<a href="https://colab.research.google.com/github/kumarchandan/END/blob/main/week9/6_Attention_AmbigQA_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### AmbigQA Dataset

https://nlp.cs.washington.edu/ambigqa/

a new open-domain question answering task that consists of predicting a set of question and answer pairs, where each plausible answer is associated with a disambiguated rewriting of the original question. A data set covering 14,042 open-ended QI-open questions.

### 3 - Neural Machine Translation by Jointly Learning to Align and Translate

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchtext
from torchtext import data
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd

import random
import math
import time

Set the random seeds for reproducability.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
TEXT = Field(tokenize='spacy', init_token='<sos>', eos_token='<eos>', lower=True)

# same number of data for question and answer so just use TEXT
fields = [('question', TEXT), ('answer', TEXT)]

Load the data.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
! cp /content/drive/MyDrive/1-Projects/END/Week9/AmbigQA/nqopen-train.json .
! cp /content/drive/MyDrive/1-Projects/END/Week9/AmbigQA/nqopen-test.json .

In [9]:
df = pd.read_json('nqopen-train.json')
df.head()

,id,question,answer
0,-6802534628745606144,total number of death row inmates in the us,"[2,718]"
1,-8178292525996414976,big little lies season 2 how many episodes,[seven]
2,1727798345024846336,who sang waiting for a girl like you,[Foreigner]
3,-1862942322519208448,where do you cross the arctic circle in norway,[Saltfjellet]
4,-6742567993987507200,who is the main character in green eggs and ham,[Sam - I - am]


Extract text from **answer** column

In [10]:
df['answer'] = df['answer'].str[0]
df.head()

,id,question,answer
0,-6802534628745606144,total number of death row inmates in the us,"2,718"
1,-8178292525996414976,big little lies season 2 how many episodes,seven
2,1727798345024846336,who sang waiting for a girl like you,Foreigner
3,-1862942322519208448,where do you cross the arctic circle in norway,Saltfjellet
4,-6742567993987507200,who is the main character in green eggs and ham,Sam - I - am


In [13]:
example = [data.Example.fromlist([df.question[i], df.answer[i]], fields=fields) for i in range(df.shape[0])]

In [14]:
amBigQADataset = data.Dataset(example, fields)
amBigQADataset, vars(amBigQADataset[0])

(<torchtext.data.dataset.Dataset at 0x7f048220e748>,
 {'answer': ['2,718'],
  'question': ['total',
   'number',
   'of',
   'death',
   'row',
   'inmates',
   'in',
   'the',
   'us']})

In [15]:
(train, valid) = amBigQADataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))
print(len(train), len(valid))

67293 11875


In [16]:
print(vars(train.examples[0]))

{'question': ['who', 'played', 'haman', 'in', 'one', 'night', 'with', 'the', 'king'], 'answer': ['james', 'callis']}


In [17]:
TEXT.build_vocab(train, min_freq = 2)

Define the device.

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Create the iterators.

In [19]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.question),
    sort_within_batch=True,
    device = device)

In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [21]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [22]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [23]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

The rest of this session is very similar to the previous one.

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [24]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [25]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(21693, 128)
    (rnn): GRU(128, 256, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (embedding): Embedding(21693, 128)
    (rnn): GRU(640, 256)
    (fc_out): Linear(in_features=896, out_features=21693, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters. We get an increase of almost 50% in the amount of parameters from the last model. 

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 26,623,037 trainable parameters


We create an optimizer.

In [27]:
optimizer = optim.Adam(model.parameters())

We initialize the loss function.

In [29]:
TEXT_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TEXT_PAD_IDX)

We then create the training loop...

In [30]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question
        trg = batch.answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [32]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question
            trg = batch.answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [34]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 46s
	Train Loss: 5.672 | Train PPL: 290.714
	 Val. Loss: 4.916 |  Val. PPL: 136.462
Epoch: 02 | Time: 0m 46s
	Train Loss: 4.825 | Train PPL: 124.596
	 Val. Loss: 4.682 |  Val. PPL: 107.983
Epoch: 03 | Time: 0m 46s
	Train Loss: 4.453 | Train PPL:  85.908
	 Val. Loss: 4.520 |  Val. PPL:  91.841
Epoch: 04 | Time: 0m 46s
	Train Loss: 4.094 | Train PPL:  59.957
	 Val. Loss: 4.439 |  Val. PPL:  84.692
Epoch: 05 | Time: 0m 46s
	Train Loss: 3.744 | Train PPL:  42.260
	 Val. Loss: 4.389 |  Val. PPL:  80.549
Epoch: 06 | Time: 0m 46s
	Train Loss: 3.369 | Train PPL:  29.039
	 Val. Loss: 4.409 |  Val. PPL:  82.175
Epoch: 07 | Time: 0m 46s
	Train Loss: 3.009 | Train PPL:  20.267
	 Val. Loss: 4.439 |  Val. PPL:  84.672
Epoch: 08 | Time: 0m 46s
	Train Loss: 2.660 | Train PPL:  14.303
	 Val. Loss: 4.525 |  Val. PPL:  92.252
Epoch: 09 | Time: 0m 46s
	Train Loss: 2.362 | Train PPL:  10.612
	 Val. Loss: 4.588 |  Val. PPL:  98.332
Epoch: 10 | Time: 0m 46s
	Train Loss: 2.100 | Train PPL

In [37]:
test_df = pd.read_json('nqopen-test.json')
test_df.head()

,id,question,answer
0,-3290814144789249536,who got the first nobel prize in physics,[Wilhelm Conrad Röntgen]
1,8851020722386421760,when is the next deadpool movie being released,"[May 18 , 2018]"
2,-8817510910624891904,which mode is used for short wave broadcast se...,"[Olivia, MFSK]"
3,955374967862684288,the south west wind blows across nigeria between,[till September]
4,-4340755100872459776,what does hp mean in war and order,[hit points or health points]


In [38]:
test_df['answer'] = test_df['answer'].str[0]
test_df.head()

,id,question,answer
0,-3290814144789249536,who got the first nobel prize in physics,Wilhelm Conrad Röntgen
1,8851020722386421760,when is the next deadpool movie being released,"May 18 , 2018"
2,-8817510910624891904,which mode is used for short wave broadcast se...,Olivia
3,955374967862684288,the south west wind blows across nigeria between,till September
4,-4340755100872459776,what does hp mean in war and order,hit points or health points


In [39]:
example_test = [data.Example.fromlist([test_df.question[i], test_df.answer[i]], fields=fields) for i in range(test_df.shape[0])]

In [40]:
amBigQADatasetTest = data.Dataset(example_test, fields)
amBigQADatasetTest

In [41]:
vars(amBigQADatasetTest[0])

{'answer': ['wilhelm', 'conrad', 'röntgen'],
 'question': ['who', 'got', 'the', 'first', 'nobel', 'prize', 'in', 'physics']}

In [45]:
test_iterator = torchtext.data.BucketIterator( # since only test data, no need of split
    amBigQADatasetTest, 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.question),
    sort_within_batch=True,
    device = device, )

In [46]:
for batch in test_iterator:
  print(batch)
  break


[torchtext.data.batch.Batch of size 128]
	[.question]:[torch.cuda.LongTensor of size 12x128 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 7x128 (GPU 0)]


In [48]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.633 | Test PPL: 102.819 |
